In [1]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
true_list = []

In [3]:
for i in range(26):
    temp = []
    a = i - 2
    b = i - 1
    c = i + 1
    d = i + 2
    temp.extend([a, b, c, d])

    # Sub-lists of input and target and 1 as label
    for j in range(4):
        if temp[j] >= 0 and temp[j] <= 25:
            true_list.append([i, temp[j], 1])
print(true_list[:5])
# [[0, 1, 1], [0, 2, 1], [1, 0, 1], [1, 2, 1], [1, 3, 1]]


[[0, 1, 1], [0, 2, 1], [1, 0, 1], [1, 2, 1], [1, 3, 1]]


In [4]:
import random


# There are 98 true pairs, keeping size 400 selects more than 300 #random pairs or roughly 3 random targets for each input
def gen_rand_list(size=400):
    # true targets are filtered here as size of vocab is too small
    false_list = []
    for i in range(size):
        frs = random.sample(range(26), 1)[0]
        sec = random.sample(range(26), 1)[0]
        if abs(frs - sec) > 2 or (frs == sec):
            false_list.append([frs, sec, 0])
    return false_list


In [7]:
def one_hot_auto():
    #Joining and shuffling true and random input/target pairs
    joint_list = np.concatenate((np.array(true_list), np.array(gen_rand_list())), axis = 0)
    np.random.shuffle(joint_list)
    inp_targ_labels = torch.Tensor(joint_list).long()
    #Converting both inputs and targets to one-hot forms 
        #Two tensors are initialized as 0 tensors
        #The item in i_th row whose index is equal to corresponding
        #input/target is then replaced by 1
    middle_word_arr = torch.zeros(inp_targ_labels.shape[0], 26)
    sur_word_arr = torch.zeros(inp_targ_labels.shape[0], 26)
    for i in range(len(inp_targ_labels)):
        middle_word_arr[i, inp_targ_labels[i, 0]] = 1
        sur_word_arr[i, inp_targ_labels[i, 1]] = 1
    labels = inp_targ_labels[:, 2].float()
    return (middle_word_arr, sur_word_arr, labels)

In [8]:
    import torch.optim as optim

    # Create Network
    # 2 fully connected layers with NO bias
    # Embedding dimension is 10
    # Rows of each weight matrix represents the embedding
    # Sigmoid is implemented using loss criterion (nn.BCELoss())
    fc_inp_word = nn.Linear(26, 10, bias=False)
    fc_targ_word = nn.Linear(26, 10, bias=False)
    LR = 0.001
    criterion = nn.BCELoss()
    params = list(fc_inp_word.parameters()) + list(fc_targ_word.parameters())
    optimizer = optim.Adam(params, lr=LR)


In [9]:
epochs = 10000
print_every = 1000
#In skip-gram middle word becomes the input which predicts #surrounding words(targets)
#Every time one_hot_auto() is called fresh batch is generated
mid_hot, sur_hot, labels = one_hot_auto()
for i in range(epochs):
#Forward prop to get hidden layer
    z_midl = fc_inp_word(torch.Tensor(mid_hot))
    z_sur = fc_targ_word(torch.Tensor(sur_hot))
    
    #Initialize a 1d matrix of 0s to store dot products between each
    #row of first hidden matrix embedding input with second hidden
    #matrix embedding target words
    #This score forms the basis for optimization
    dot_u_v = torch.zeros(mid_hot.shape[0], 1)
    for j in range(len(z_midl)):
        dot_u_v[j, :] = z_midl[j, :] @ z_sur[j, :]
        
    #Sigmoid activation applied to dot products of vectors
    desired_logits = dot_u_v
    sig_logits = nn.Sigmoid()(desired_logits)
    
    #Back prop and stepping
    optimizer.zero_grad()
    loss = criterion(sig_logits, torch.Tensor(labels).view(sig_logits.shape[0], 1))
    loss.backward()
    optimizer.step()
    if i % print_every == 0:
        print(loss.data)
        #Scheduled one_hot_auto() to generate fresh random pairs 
        mid_hot, sur_hot, labels = one_hot_auto()

tensor(0.6934)
tensor(0.0798)
tensor(0.0172)
tensor(0.0081)
tensor(0.0039)
tensor(0.0025)


KeyboardInterrupt: 